<a href="https://colab.research.google.com/github/MarcPal08/2024-bracciano-iris/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create dataset

In [6]:
!pip install transformers datasets

In [8]:
from datasets import load_dataset

In [9]:
v01_files = ["v01/m00 - m00-1.csv",
    "v01/m00 - m00-2.csv",
    "v01/m00 - m00-3.csv",
    "v01/m00 - m00-4.csv",
    "v01/m01 - m01-1.csv",
    "v01/m02 - m02-1.csv",
    "v01/m03 - m03-1.csv",
    "v01/m04 - m04-1.csv"]
dataset = load_dataset("csv",data_files=v01_files)

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 349
    })
})

In [12]:
sample = dataset["train"].shuffle(seed=40).select(range(10))
print(sample[:3])

{'text': ['Sono importanti perché utili alle competenze lavorative necessarie', 'Ottima la dimensione didattica, mediocre la dimensione organizzativa e di coordinamento.', 'la scuola mi sta aiutando a crescere formalmente e mi sta insegnando cose importanti per la mia vita'], 'sentiment': ['positive', 'positive', 'positive']}


In [ ]:
dataset_clean = dataset["train"].train_test_split(train_size=0.8, seed=42)

In [ ]:
dataset_clean.save_to_disk('v01/dataset_clean')

# Upload dataset

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset_clean.push_to_hub("sentiment-analysis-test")

# Tokenize it!

In [ ]:
from transformers import AutoTokenizer
from datasets import load_from_disk

dataset = load_from_disk("v01/dataset_clean")

tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment")



In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)